<a href="https://colab.research.google.com/github/cherishkim42/final-project/blob/master/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💥 Final Project: Manipulate a DataFrame and train ML/DL model using PySpark 💥

_Thanks to what I believe to be Anaconda config issues, I was unable to install and work with the current version of PySpark using Anaconda/Jupyter. I was able to circumvent this blocker by using Google Colab instead! Woot Woot_

In [2]:
|!pip install pyspark==3.0.0

     |████████████████████████████████| 204.7MB 71kB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044184 sha256=f386ffb7a88cb5393aa9b6ae7cfc7f27739a768681268f055d15bf83dd8e5a5c
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [3]:
import pyspark

pyspark.__version__

'3.0.0'

⚡️ Instantiate a SparkSession.

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Titanic Data').getOrCreate()
spark

### 👀 Quick look at the training data

In [8]:
df_train = (spark.read.format("csv").option("header", "true").load("train.csv"))
df_train.take(5)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S'),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C'),
 Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin=None, Embarked='S'),
 Row(PassengerId='4', Survived='1', Pclass='1', Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35', SibSp='1', Parch='0', Ticket='113803', Fare='53.1', Cabin='C123', Embarked='S'),
 Row(PassengerId='5', Survived='0', Pclass='3', Name='Allen, Mr. William Henry', Sex='male', Age='35', SibSp='0', Parch='0', Ticket='373450', Fare='8.05', Cabin=None, Embarked='S')]

In [9]:
df_train.show(5)
df_train.limit(5).toPandas()

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,None,S


In [10]:
df_train.count()

891

In [12]:
df_train.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [14]:
df_train.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [15]:
df_train.describe()

DataFrame[summary: string, PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [16]:
df_train.describe().toPandas()

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0,A10,C
4,max,99,1,3,"van Melkebeke, Mr. Philemon",male,9,8,6,WE/P 5735,93.5,T,S


### 🧼 Sanitize the data

Imports first!

In [17]:
from pyspark.sql.functions import isnull, when, count, col
from pyspark.ml.feature import StringIndexer, VectorAssembler

In [18]:
df_train = df_train.select(col('Survived').cast('float'),
                           col('Pclass').cast('float'),
                           col('Sex'), col('Age').cast('float'),
                           col('Fare').cast('float'), col('Embarked'))
df_train.show()

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|   8.05|       S|
|     0.0|   3.0|  male|null| 8.4583|       Q|
|     0.0|   1.0|  male|54.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0| 21.075|       S|
|     1.0|   3.0|female|27.0|11.1333|       S|
|     1.0|   2.0|female|14.0|30.0708|       C|
|     1.0|   3.0|female| 4.0|   16.7|       S|
|     1.0|   1.0|female|58.0|  26.55|       S|
|     0.0|   3.0|  male|20.0|   8.05|       S|
|     0.0|   3.0|  male|39.0| 31.275|       S|
|     0.0|   3.0|female|14.0| 7.8542|       S|
|     1.0|   2.0|female|55.0|   16.0|       S|
|     0.0|   3.0|  male| 2.0| 29.125|       Q|
|     1.0|   2.0|  male|null|   13.0|       S|
|     0.0|   

In [20]:
df_train.select([count(when(isnull(c), c)).alias(c) for c in df_train.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



Remove problematic, null-y data.

In [21]:
df_train = df_train.replace('?', None).dropna(how='any')

We want to work with the data from columns "Sex" and "Embarked" -- there's useful information to be gleaned from here, but PySpark's ML packages can only use numeric data. That's where **one-hot encoding** steps in, with `StringIndexer` from the assortment of PySpark ML imports.

In [22]:
df_train = StringIndexer(inputCol="Sex", outputCol="Gender", handleInvalid="keep").fit(df_train).transform(df_train)
df_train = StringIndexer(inputCol="Embarked", outputCol="Boarded", handleInvalid="keep").fit(df_train).transform(df_train)
df_train.take(5)

[Row(Survived=0.0, Pclass=3.0, Sex='male', Age=22.0, Fare=7.25, Embarked='S', Gender=0.0, Boarded=0.0),
 Row(Survived=1.0, Pclass=1.0, Sex='female', Age=38.0, Fare=71.2833023071289, Embarked='C', Gender=1.0, Boarded=1.0),
 Row(Survived=1.0, Pclass=3.0, Sex='female', Age=26.0, Fare=7.925000190734863, Embarked='S', Gender=1.0, Boarded=0.0),
 Row(Survived=1.0, Pclass=1.0, Sex='female', Age=35.0, Fare=53.099998474121094, Embarked='S', Gender=1.0, Boarded=0.0),
 Row(Survived=0.0, Pclass=3.0, Sex='male', Age=35.0, Fare=8.050000190734863, Embarked='S', Gender=0.0, Boarded=0.0)]

In [23]:
df_train.show(5)

+--------+------+------+----+-------+--------+------+-------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
|     1.0|   3.0|female|26.0|  7.925|       S|   1.0|    0.0|
|     1.0|   1.0|female|35.0|   53.1|       S|   1.0|    0.0|
|     0.0|   3.0|  male|35.0|   8.05|       S|   0.0|    0.0|
+--------+------+------+----+-------+--------+------+-------+
only showing top 5 rows



In [24]:
df_train.dtypes

[('Survived', 'float'),
 ('Pclass', 'float'),
 ('Sex', 'string'),
 ('Age', 'float'),
 ('Fare', 'float'),
 ('Embarked', 'string'),
 ('Gender', 'double'),
 ('Boarded', 'double')]

In [25]:
df_train.describe().toPandas()

,summary,Survived,Pclass,Sex,Age,Fare,Embarked,Gender,Boarded
0,count,712,712,712,712,712,712,712,712
1,mean,0.4044943820224719,2.240168539325843,None,29.642092696610796,34.567251479357815,None,0.3637640449438202,0.2612359550561798
2,stddev,0.4911389472541192,0.8368543166903446,None,14.492932900360593,52.93864884500567,None,0.4814200635414358,0.5215606143930027
3,min,0.0,1.0,female,0.42,0.0,C,0.0,0.0
4,max,1.0,3.0,male,80.0,512.3292,S,1.0,2.0


In [27]:
df_train = df_train.drop("Sex", "Embarked")
df_train.show(5)

+--------+------+----+-------+------+-------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|
+--------+------+----+-------+------+-------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|
+--------+------+----+-------+------+-------+
only showing top 5 rows



Aaaand a final bit of data wrangling to get our dataframe to work with PySpark:

In [29]:
pyspark_features = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
vector_assembler = VectorAssembler(inputCols=pyspark_features, outputCol='features')
df_train = vector_assembler.transform(df_train)
df_train.show(5)

+--------+------+----+-------+------+-------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|            features|
+--------+------+----+-------+------+-------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+-------+--------------------+
only showing top 5 rows



In [36]:
(df_train, df_test) = df_train.randomSplit([0.8, 0.2])

In [38]:
df_train.show(5)
df_test.show(5)

+--------+------+----+--------+------+-------+--------------------+
|Survived|Pclass| Age|    Fare|Gender|Boarded|            features|
+--------+------+----+--------+------+-------+--------------------+
|     0.0|   1.0| 2.0|  151.55|   1.0|    0.0|[1.0,2.0,151.5500...|
|     0.0|   1.0|18.0|   108.9|   0.0|    1.0|[1.0,18.0,108.900...|
|     0.0|   1.0|19.0|    53.1|   0.0|    0.0|[1.0,19.0,53.0999...|
|     0.0|   1.0|21.0| 77.2875|   0.0|    0.0|[1.0,21.0,77.2874...|
|     0.0|   1.0|22.0|135.6333|   0.0|    1.0|[1.0,22.0,135.633...|
+--------+------+----+--------+------+-------+--------------------+
only showing top 5 rows

+--------+------+----+-----+------+-------+--------------------+
|Survived|Pclass| Age| Fare|Gender|Boarded|            features|
+--------+------+----+-----+------+-------+--------------------+
|     0.0|   1.0|19.0|263.0|   0.0|    0.0|[1.0,19.0,263.0,0...|
|     0.0|   1.0|28.0| 47.1|   0.0|    0.0|[1.0,28.0,47.0999...|
|     0.0|   1.0|31.0| 52.0|   0.0|   

### 💃 It is model time!

Imports first

In [39]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
random_forest = RandomForestClassifier(labelCol="Survived", featuresCol='features', maxDepth=5)
model = random_forest.fit(df_train)
predictions = model.transform(df_test)

In [42]:
model_evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
model_evaluation = model_evaluator.evaluate(predictions)
print("Titanic Survivor Model Accuracy: ", model_evaluation)

Titanic Survivor Model Accuracy:  0.7777777777777778
